In [1]:
%matplotlib inline
from IPython import display
from matplotlib import pyplot as plt
import os
import imageio
import requests
from ipywidgets import Image
import ipywidgets

import datajoint as dj

In [2]:
## Storage configuration 
# set up stores
dj.config['stores'] = {
    'local': {  # store in files
        'protocol': 'file',
        'location': os.path.abspath('./dj-store')
    }}

In [3]:
schema = dj.schema('common_attach')

Please enter DataJoint username: root
Please enter DataJoint password: ········
Connecting root@localhost:3306


In [4]:
@schema
class WebImage(dj.Lookup):
    definition = """
    # A reference to a web image
    image_number : int
    ---
    image_name : varchar(30)
    image_description : varchar(1024)
    image_url : varchar(1024)
    
    unique index(image_name)
    """
    contents = [
        (0, "pyramidal", 
         
         'Coronal section containing the chronically imaged pyramidal neuron "dow" '\
         '(visualized by green GFP) does not stain for GABA (visualized by antibody staining in red). '\
         'Confocal image stack, overlay of GFP and GABA channels. Scale bar: 100 um',
         
         "https://upload.wikimedia.org/wikipedia/commons/d/dc/PLoSBiol4.e126.Fig6fNeuron.jpg"
        ),
        (1, "striatal", 
         
         "Mouse spiny striatal projection neuron expressing a transgenic fluorescent protein "\
         "(colored yellow) delivered by a recombinant virus (AAV). "\
         "The striatal interneuron are stainerd in green for the neurokinin-1 receptor.",
         
         "https://upload.wikimedia.org/wikipedia/commons/e/e8/Striatal_neuron_in_an_interneuron_cage.jpg"
        )
    ]

In [5]:
Image.from_url((WebImage & 'image_number=0').fetch1('image_url'))

Image(value=b'https://upload.wikimedia.org/wikipedia/commons/d/dc/PLoSBiol4.e126.Fig6fNeuron.jpg', format='url…

In [9]:
@schema
class OriginalFile(dj.Imported):
    definition = """
    -> WebImage
    ---
    image_file : attach@local
    """
    
    def make(self, key):
        # get the URL
        url = (WebImage & key).fetch1('image_url')
        
        # download the file from the web
        local_file = os.path.join(os.path.abspath('.'), url.split('/')[-1])
        with open(local_file, 'wb') as f:
            f.write(requests.get(url).content)
            
        # attach the file
        self.insert1(dict(key, image_file=local_file))
        
        # delete the downloaded file
        os.remove(local_file)

In [10]:
# perform the download
OriginalFile.populate()

In [11]:
OriginalFile()

image_number,image_file
0,=BLOB=
1,=BLOB=


In [12]:
# Declare a table with a configurable blob
@schema
class Slide(dj.Computed):
    definition = """
    -> OriginalFile
    ---
    image_array : blob@local  # array in specified store
    """
    
    def make(self, key):
        # get the attached file
        file = (OriginalFile & key).fetch1('image_file')
        
        # save image data
        self.insert1(dict(key, image_array=imageio.imread(file)))
        
        # remove the downloaded file
        os.remove(file)

In [13]:
Slide.populate()

In [16]:
Slide()

image_number,image_array array in specified store
1,=BLOB=
0,=BLOB=


In [17]:
OriginalFile()

image_number,image_file
0,=BLOB=
1,=BLOB=


In [22]:
schema.external['local']

hash,size size of object in bytes,timestamp automatic timestamp
00bffedc-d5c2-f928-f0d5-4d31fe01311d,806164,2019-08-01 09:06:33
403b8717-6c83-fa52-d381-e1371e61e4ac,1324311,2019-08-01 09:06:55
81c1c2d3-3c08-66ed-1d64-c96376539195,895094,2019-08-01 09:06:33
cbc9f232-6abd-6863-c449-9cb82a11e47a,1707130,2019-08-01 09:06:55


In [23]:
schema.external['local'].delete()

Deleted 1 items


In [21]:
(Slide & 'image_number = 1').delete()

About to delete:
`common_attach`.`__slide`: 1 items
Proceed? [yes, No]: yes
Committed.


In [26]:
schema.external['local'].clean()

Deleting...
Deleted 0 objects
